In [3]:
#!/usr/bin/env python3

import os
import sys
import threading
import cv2
import numpy as np
import yaml
import pickle as pkl
import pdb
from tqdm import tqdm
from joblib import Parallel,delayed

def get_surf_features_from_video(surf,video_name,keyframe_interval):
    "Receives filename of downsampled video and of output path for features. Extracts features in the given keyframe_interval. Saves features in pickled file."
    # TODO
    downsampled_video_filename = os.path.join(downsampled_videos, video_name + '.mp4')
#         print(downsampled_video_filename)
    surf_feat_video_filename = os.path.join(surf_features_folderpath, video_name + '.surf')
    frames = get_keyframes(downsampled_video_filename,keyframe_interval)
    features = []
    max_len = 0
    for keyframe in frames:
        key, des = surf.detectAndCompute(keyframe,None)
        features.append(des)

    with open(surf_feat_video_filename,"wb") as o:
        pkl.dump(features,o)
        

def get_keyframes(downsampled_video_filename, keyframe_interval):
    "Generator function which returns the next keyframe."

    # Create video capture object
    video_cap = cv2.VideoCapture(downsampled_video_filename)
    frame = 0
    while True:
        frame += 1
        ret, img = video_cap.read()
        if ret is False:
            break
        if frame % keyframe_interval == 0:
            yield img
    video_cap.release()

In [4]:
print("Beginning SURF extraction")

all_video_names = "list/all.video"
config_file = "config.yaml"
my_params = yaml.load(open(config_file))

# Get parameters from config file
keyframe_interval = my_params.get('keyframe_interval')
hessian_threshold = my_params.get('hessian_threshold')
surf_features_folderpath = my_params.get('surf_features')
downsampled_videos = my_params.get('downsampled_videos')

# TODO: Create SURF object
surf = cv2.SURF(hessian_threshold)

# Check if folder for SURF features exists
if not os.path.exists(surf_features_folderpath):
    os.mkdir(surf_features_folderpath)

# Loop over all videos (training, val, testing)
# TODO: get SURF features for all videos but only from keyframes

fread = open(all_video_names, "r")

# for line in tqdm(fread.readlines()):
#     video_name = line.replace('\n', '')

#     downsampled_video_filename = os.path.join(downsampled_videos, video_name + '.mp4')
# #         print(downsampled_video_filename)
#     surf_feat_video_filename = os.path.join(surf_features_folderpath, video_name + '.surf')

#     if not os.path.isfile(downsampled_video_filename):
# #             print("here")
#         continue

#     # Get SURF features for one video
#     get_surf_features_from_video(surf,downsampled_video_filename,
#                                  surf_feat_video_filename, keyframe_interval)

Beginning SURF extraction


AttributeError: module 'cv2' has no attribute 'SURF'

In [2]:
with Parallel(n_jobs=4) as parallel:
    res = parallel(delayed(get_surf_features_from_video)(surf,video_name,keyframe_interval) for video_name in tqdm(fread.readlines()))

NameError: name 'Parallel' is not defined